In [14]:
import requests
import os
from bs4 import BeautifulSoup
from tqdm import tqdm

In [19]:
set_numbers = [f'0{i}' for i in range(1,8)]
card_numbers = [f'{i:03}' for i in range(1,120)]

In [56]:
base_url = 'https://image.optcg.gg/images/en/'

os.mkdir('C:\\Users\\Wayne Ewing\\Pictures\\card images\\')

In [54]:
save_path = 'C:\\Users\\Wayne Ewing\\Pictures\\optcg_images'
save_path


'C:\\Users\\Wayne Ewing\\Pictures\\optcg_images'

In [68]:
os.mkdir('C:\\Users\\Wayne Ewing\\Pictures\\card images\\digimontcg_images')
digimon_url = 'https://images.digimoncard.io/images/cards/'
digi_save_path = 'C:\\Users\\Wayne Ewing\\Pictures\\card images\\digimontcg_images'

In [62]:
digi_set = [f'{i}' for i in range(1,8)]
digi_numbers = [f'{i:03}' for i in range(1,113)]

In [69]:
for set_num in digi_set:
    for card_num in tqdm(digi_numbers):
        image_url = f'{digimon_url}BT{set_num}-{card_num}.jpg'
    
        try:

            response = requests.get(image_url)
            response.raise_for_status()

            image_path = os.path.join(digi_save_path,f'BT{set_num}-{card_num}.png')

            if response.status_code == 200:
                with open(image_path, 'wb') as f:
                    f.write(response.content)
            else:
                print(f"Image not found or failed to download: {image_url} (Status code: {response.status_code})")

        except requests.exceptions.HTTPError as err:
            print(f'Image not found: {image_url}')

100%|██████████| 112/112 [00:05<00:00, 21.29it/s]
